In [1]:
import os
import re
from collections import defaultdict

import textract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from PyPDF2 import PdfReader, PdfWriter
from tqdm.autonotebook import tqdm
import pandas as pd

/var/folders/4_/r6xyhscs2mz_v_3y2r71_r2h0000gn/T/ipykernel_16323/945261966.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
RE_DERIVED = re.compile(r"\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile(r"[а-яА-Яa-zA-Z0-9\-]+")

stopwords_ru = stopwords.words("russian")

In [3]:
morph = MorphAnalyzer()
cp_names = {
    "ак",
    "акб",
    "акига",
    "алмазэргиэнбанк",
    "алрос",
    "альфабанк",
    "алюминиевый",
    "амвэя",
    "ангарский",
    "анк",
    "ао",
    "архангельский",
    "ас",
    "асэ",
    "атомпроект",
    "атомредметзолото",
    "атомэнергомаш",
    "атомэнергопроект",
    "атомэнергопром",
    "афк",
    "ашан",
    "аэрофлот",
    "балтика",
    "барс",
    "бат",
    "башнефть",
    "билайн",
    "бифид",
    "бф",
    "виктория",
    "владивостокский",
    "внипиэт",
    "волга",
    "волгателек",
    "волжский",
    "восток",
    "втб",
    "выксунский",
    "вымпелком",
    "вэб",
    "газпром",
    "газпромбанк",
    "гарантинвест",
    "генерировать",
    "геннадий",
    "гидропресс",
    "гк",
    "гмк",
    "гнц",
    "головной",
    "гольцблат",
    "грэс",
    "гтлк",
    "губкин",
    "дания",
    "дивизион",
    "евразийский",
    "еврохим",
    "елена",
    "енисейский",
    "еэс",
    "желдортранс",
    "зао",
    "зарубежнефть",
    "ик",
    "илим",
    "имииафрикантовый",
    "инжиниринговый",
    "инккапитал",
    "интер",
    "интернэшнл",
    "информация",
    "иркутск",
    "казаньоргсинтез",
    "казмунайгаз",
    "калининградский",
    "камаз",
    "касперский",
    "кб",
    "кемикалс",
    "ковровский",
    "комстаротс",
    "красцветмет",
    "крафтфудс",
    "кубань",
    "кузбассэнерго",
    "ленэнерго",
    "леруа",
    "лпк",
    "лср",
    "лукойл",
    "мвидео",
    "мдмбанк",
    "мегафон",
    "мерлен",
    "металлоинвест",
    "металлургический",
    "метафракс",
    "ммк",
    "монди",
    "мон’дэлиса",
    "моррис",
    "мосводоканал",
    "московский",
    "мрск",
    "мрф",
    "мсп",
    "мтс",
    "мхк",
    "неманский",
    "нестля",
    "ниаэп",
    "нижнекамскнефтехим",
    "нииар",
    "нипигаз",
    "ниу",
    "нк",
    "нлмк",
    "новатэк",
    "новогор",
    "новосибирский",
    "нордиск",
    "норильский",
    "оао",
    "объединение",
    "огк",
    "окб",
    "окбм",
    "около",
    "омк",
    "они",
    "ооо",
    "пао",
    "пейп",
    "пепеляев",
    "петропавловск",
    "пивоваренный",
    "пивоваров",
    "пигмент",
    "пик",
    "по",
    "полиметалл",
    "приволжье",
    "прикамье",
    "рао",
    "распадский",
    "ргу",
    "реновый",
    "ржд",
    "рксхолдинг",
    "рольф",
    "росатом",
    "росводоканал",
    "роснефть",
    "россеть",
    "российский",
    "россия",
    "ростелеком",
    "ростёха",
    "росэнергоатом",
    "рудна",
    "русало",
    "русгидро",
    "руссинвест",
    "руссия",
    "русый",
    "русь",
    "рф",
    "рязанский",
    "сабмиллер",
    "санофироссия",
    "сахалин",
    "сбербанк",
    "свез",
    "севергазпром",
    "севернефтегазпром",
    "северсталь",
    "сзлк",
    "сибирь",
    "сибурхолдинг",
    "система",
    "ситибанк",
    "ситроникс",
    "совкомбанк",
    "спбаэп",
    "суал",
    "сургутнефтегаз",
    "суэк",
    "схк",
    "сыктывкарский",
    "такед",
    "таманьнефтегаз",
    "татнефть",
    "твэл",
    "тгк",
    "техснабэкспорт",
    "тимченко",
    "тмк",
    "тнк",
    "тнквр",
    "тоаз",
    "трансаэро",
    "трансгаз",
    "транснефть",
    "тюмень",
    "урал",
    "уралкалий",
    "уралсиб",
    "уральский",
    "урсабанк",
    "ухта",
    "фгк",
    "ферреро",
    "фиабанк",
    "филиал",
    "филип",
    "фк",
    "фортум",
    "фосагро",
    "фпк",
    "фск",
    "химконцентрат",
    "центринвест",
    "черкизовый",
    "чусовской",
    "шелл",
    "шереметьево",
    "щуровский",
    "эксонмобила",
    "энела",
    "энергохолдинг",
    "энерджи",
    "эсэфай",
    "южнороссийский",
    "юкос",
    "юникредитбанк",
    "юнипро",
    "яндекс",
    "ятэк",
}

In [4]:
def parse_pages_to_paragraphs(files: list[str], rep_year):
    errors = []
    df = []
    report_idx = 0
    for report_name in tqdm(files):
        if not report_name.endswith(".txt"):
            print(report_name)
            continue
        try:
            report = textract.process(report_name)
        except Exception as e:
            errors.append((report_name, e))
            continue
        
        #company_name = report_name[:-4].split("_")[0]
        company_name = report_name.split("/")[-1]
        company_name = company_name[:-4].split("_")[0]
        print(company_name)
        if report_name[:-4].split("_")[-1].isnumeric():
            report_index = int(report_name[:-4].split("_")[-1])
        else:
            file_name = report_name.split("/")[-1]
            print(file_name)
            try:
                report_index = int(report_name[:-4].split("$")[-1])
                company_name = company_name.split("$")[0]
                print(report_index)
                print(company_name)
                #report_index = info_df.loc[info_df["ссылка на отчет"]==file_name].index[0]
            except:
                file_name = report_name.split("/")[-1]
                print(file_name)
                try:
                    report_index = int(report_name[:-4].split("»")[-1])
                    company_name = company_name.split("»")[0]
                    print(report_index)
                    print(company_name)
                    
                except Exception as e:
                    # broken beeline
                    print(e)
                    continue

        for i, paragraph in enumerate(report.decode().split("\n")):
            if len(paragraph)>200:
                paragraph = paragraph.replace("\n", " ").lower()
                report_page = RE_DERIVED.sub("", paragraph)
                report_page_lst = word_tokenize(report_page)
                # russian_report_page_lst = [w for w in filter(RE_RUSSIAN_TEXT.match, report_page_lst)]
                tokens = []
                # nouns = set()
                # verbs = set()
                # adj = set()
                for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
                    if word_ and word_ not in stopwords_ru and word_ not in cp_names:
                        word_ = word_.strip()
                        word_ = morph.parse(word_)[0]
                        normal_form = word_.normal_form
                        tokens.append(normal_form)
                        # if word_.tag.POS in NOUNS:
                        #     nouns.add(normal_form)
                        # elif word_.tag.POS in VERBS:
                        #     verbs.add(normal_form)
                        # elif word_.tag.POS in ADJ:
                        #     adj.add(normal_form)
                # if len(tokens) == 0:
                #     continue
                print(report_index, ' ', company_name)

                df.append(
                    {
                    "rsspp_index": report_index,
                    "index": report_index,
                    "company": company_name,
                    "paragraph": i,
                    "original_text": paragraph,
                    "cleaned_text": " ".join(tokens),
                    "year": rep_year
                    }
                )
    print(*errors, sep="\n")
    return df

In [5]:
REPORTS_DIRECTORY = "reports/2022/txt_2022"
files = [f"{REPORTS_DIRECTORY}/{file}" for file in os.listdir(REPORTS_DIRECTORY)]

In [6]:
df = parse_pages_to_paragraphs(files, 2022)

  0%|          | 0/22 [00:00<?, ?it/s]

ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 

1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-

1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-Энерго 2022
1428   ЭЛ5-

1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акрон 2022
1420   Акр

...

319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК

319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК

319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК

319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК «Норильский никель
319   ПАО «ГМК

4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus
4   Polyus

In [7]:
new_df = pd.DataFrame(df)
new_df

,rsspp_index,index,company,paragraph,original_text,cleaned_text,year
0,1428,1428,ЭЛ5-Энерго 2022,42,алибек айбекович тналин генеральный директор э...,алибек айбекович тналин генеральный директор у...,2022
1,1428,1428,ЭЛ5-Энерго 2022,43,"в завершение хочу еще раз отметить, что данные...",завершение хотеть отметить дать достижение явл...,2022
2,1428,1428,ЭЛ5-Энерго 2022,47,представляем вашему вниманию наш второй отчет ...,представлять ваш внимание наш второй отчёт уст...,2022
3,1428,1428,ЭЛ5-Энерго 2022,48,наша гордость в 2022 году достижение показател...,наш гордость 2022 год достижение показатель но...,2022
4,1428,1428,ЭЛ5-Энерго 2022,50,мы прилагаем большие усилия для развития и мот...,прилагать больший усилие развитие мотивация со...,2022
...,...,...,...,...,...,...,...
14149,1421,1421,ВК 2022,25,с помощью этого инструмента можно перевести вс...,помощь инструмент перевести ключевой кадровый ...,2022
14150,1421,1421,ВК 2022,26,году. после полного подключения к системе кэдо...,год полный подключение система кэдый этот комп...,2022
14151,1421,1421,ВК 2022,27,об отчете отчет опубликован на русском и англи...,отчёт отчёт опубликовать русский английский яз...,2022
14152,1421,1421,ВК 2022,30,vk крупнейшая российская технологическая комп...,vk крупный российский технологический компания...,2022


In [8]:
new_df.to_csv("paragraphs/paragraphs_2022.csv")